# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 68 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.02670


extracting tarball to tmp_2211.02670...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.02673


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.02670/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'institutions.tex' from 'tmp_2211.02670/institutions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'institutions.tex' from 'tmp_2211.02670/Flashlights Initial Results(1)/institutions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure fig/gtcomparison
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure fig/gtcomparisoncalib
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure ./0_Figures/CM_G_G-RP_Pleiades_Abs
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))


Retrieving document from  https://arxiv.org/e-print/2211.03747


extracting tarball to tmp_2211.03747... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03641-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03641) | **Ultracool dwarfs in Gaia DR3**  |
|| L.M. Sarro, et al. -- incl., <mark>M. García-Torres</mark>, <mark>C.A.L. Bailer-Jones</mark>, <mark>R. Andrae</mark>, <mark>M. Fouesneau</mark>, <mark>E. Livanou</mark>, <mark>H. Zhao</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| *Accepted by Astronomy and Astrophysics. 29 pages, 20 figures plus 3 appendices*|
|**Abstract**| Aims. In this work we use the Gaia DR3 set of ultracool dwarf candidates and complement the Gaia spectrophotometry with additional photometry in order to characterise its global properties. This includes the inference of the distances, their locus in the Gaia colour-absolute magnitude diagram and the (biased through selection) luminosity function in the faint end of the Main Sequence. We study the overall changes in the Gaia RP spectra as a function of spectral type. We study the UCDs in binary systems, attempt to identify low-mass members of nearby young associations, star forming regions and clusters, and analyse their variability properties. Results. We detect 57 young, kinematically homogeneous groups some of which are identified as well known star forming regions, associations and clusters of different ages. We find that the primary members of 880 binary systems with a UCD belong mainly to the thin and thick disk components of the Milky Way. We identify 1109 variable UCDs using the variability tables in the Gaia archive, 728 of which belong to the star forming regions defined by HMAC. We define two groups of variable UCDs with extreme bright or faint outliers. Conclusions. The set of sources identified as UCDs in the Gaia archive contains a wealth of information that will require focused follow-up studies and observations. It will help to advance our understanding of the nature of the faint end of the Main Sequence and the stellar/substellar transition. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03416-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03416) | **Comparative analysis of atmospheric parameters from high-resolution  spectroscopic sky surveys: APOGEE, GALAH, Gaia-ESO**  |
|| <mark>Viola Hegedűs</mark>, et al. -- incl., <mark>Domingo Aníbal García-Hernández</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| *Accepted in A&A, 23 pages, 13 figures, 6 tables*|
|**Abstract**| SDSS-IV APOGEE-2, GALAH and Gaia-ESO are high resolution, ground-based, multi-object spectroscopic surveys providing fundamental stellar atmospheric parameters and multiple elemental abundance ratios for hundreds of thousands of stars of the Milky Way. We undertake a comparison between the most recent data releases of these surveys to investigate the accuracy and precision of derived parameters by placing the abundances on an absolute scale. We discuss the correlations in parameter and abundance differences as a function of main parameters. Uncovering the variants provides a basis to on-going efforts of future sky surveys. Quality samples from the APOGEE-GALAH, APOGEE-GES and GALAH-GES overlapping catalogs are collected. We investigate the mean variants between the surveys, and linear trends are also investigated. We compare the slope of correlations and mean differences with the reported uncertainties. The average and scatter of vrad, Teff, log g, [M/H] and vmicro, along with numerous species of elemental abundances in the combined catalogs show that in general there is a good agreement between the surveys. We find large radial velocity scatters ranging from 1.3 km/s to 4.4 km/s when comparing the three surveys. We observe weak trends: e.g. in $\Delta$Teff vs. $\Delta$log g for the APOGEE-GES stars, and a clear correlation in the vmicro-$\Delta$vmicro planes in the APOGEE-GALAH common sample. For [$\alpha$/H], [Ti/H] (APOGEE-GALAH giants) and [Al/H] (APOGEE-GALAH dwarfs) potential strong correlations are discovered as a function of the differences in the main atmospheric parameters, and we find weak trends for other elements. In general we find good agreement between the three surveys within their respective uncertainties. However, there are certain regimes in which strong variants exist, which we discuss. There are still offsets larger than 0.1 dex in the absolute abundance scales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02673-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02673) | **Unusual Abundances from Planetary System Material Polluting the White  Dwarf G238-44**  |
|| Ted M Johnson, et al. -- incl., <mark>Beth L. Klein</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| *Accepted to ApJ. 18 pages, 9 figures, 7 tables*|
|**Abstract**| Ultraviolet and optical spectra of the hydrogen-dominated atmosphere white dwarf star G238-44 obtained with FUSE, Keck/HIRES, HST/COS, and HST/STIS reveal ten elements heavier than helium: C, N, O, Mg, Al, Si, P, S, Ca, and Fe. G238-44 is only the third white dwarf with nitrogen detected in its atmosphere from polluting planetary system material. Keck/HIRES data taken on eleven nights over 24 years show no evidence for variation in the equivalent width of measured absorption lines, suggesting stable and continuous accretion from a circumstellar reservoir. From measured abundances and limits on other elements we find an anomalous abundance pattern and evidence for the presence of metallic iron. If the pollution is from a single parent body, then it would have no known counterpart within the solar system. If we allow for two distinct parent bodies, then we can reproduce the observed abundances with a mix of iron-rich Mercury-like material and an analog of an icy Kuiper Belt object with a respective mass ratio of 1.7:1. Such compositionally disparate objects would provide chemical evidence for both rocky and icy bodies in an exoplanetary system and would be indicative of a planetary system so strongly perturbed that G238-44 is able to capture both asteroid- and Kuiper Belt-analog bodies near-simultaneously within its $<$100 Myr cooling age. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03414-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03414) | **Constraints on axionlike particles from a combined analysis of three  flaring $\textit{Fermi}$ flat-spectrum radio quasars**  |
|| <mark>James Davies</mark>, <mark>Manuel Meyer</mark>, Garret Cotter |
|*Appeared on*| *2022-11-08*|
|*Comments*| *18 pages, 14 figures; submitted to Physical Review D*|
|**Abstract**| Many theories beyond the Standard Model of particle physics predict the existence of axionlike particles (ALPs) that mix with photons in the presence of a magnetic field. Searching for the effects of ALP-photon mixing in gamma-ray observations of blazars has provided some of the strongest constraints on ALP parameter space so far. Previously, only individual sources have been analysed. We perform a combined analysis on $\textit{Fermi}$ Large Area Telescope data of three bright, flaring flat-spectrum radio quasars, with the blazar jets themselves as the dominant mixing region. For the first time, we include a full treatment of photon-photon dispersion within the jet, and account for the uncertainty in our B-field model by leaving the field strength free in the fitting. Overall, we find no evidence for ALPs, but are able to exclude the ALP parameters $m_a\lesssim200$ neV and $g_{a\gamma}\gtrsim 5 \times 10^{-12}$ GeV$^{-1}$ with 95\% confidence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03557-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03557) | **Shape asymmetries and lopsidedness-radial-alignment in simulated  galaxies**  |
|| Jinzhi Shen, Xufen Wu, Yirui Zheng, <mark>Beibei Guo</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| *16 pages, 12 figures, 3 tables, submitted to MNRAS*|
|**Abstract**| Galaxies are observed to be lopsided, meaning that they are more massive and more extended along one direction than the opposite. However, the galaxies generated in cosmological simulations are much less lopsided, inconsistent with observations. In this work, we provide a statistical analysis of the lopsided morphology of 2148 simulated isolated satellite galaxies generated by TNG50-1 simulation, incorporating the effect of tidal fields from halo centres. We study the radial alignment (RA) between the major axes of satellites and the radial direction of their halo centres within truncation radii of $3R_h$, $5R_h$ and $10R_h$. According to our results, RA is absent for all these truncations. We also calculate the far-to-near-side semi-axial ratios of the major axes, denoted by $a_-/a_+$, which measures the semi-axial ratios of the major axes in the hemispheres between backwards (far-side) and facing (near-side) the halo centres. If the satellites are truncated within radii of $3R_h$ and $5R_h$ with $R_h$ being the stellar half mass radius, the numbers of satellites with longer semi-axes on the far-side are found to be almost equal to those with longer semi-axes on the near-side. Within a larger truncated radius of $10R_h$, the number of satellites with axial ratios $a_-/a_+ <1.0$ is about $10\%$ more than that with $a_-/a_+ > 1.0$. Therefore, the tidal fields from halo centres play a minor role in the generation of lopsided satellites. The lopsidedness radial alignment (LRA), i.e., an alignment of long semi-major-axes along the radial direction of halo centres, is further studied. No clear evidence of LRA is found in our sample within the framework of $\Lambda$CDM Newtonian dynamics. In comparison, the LRA can be naturally induced by the external fields from the central host galaxy in Milgromian dynamics. (See paper for full abstract) |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03582-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03582) | **55 Cancri e's occultation captured with CHEOPS**  |
|| B.-O. Demory, et al. -- incl., <mark>S. Hoyer</mark>, <mark>K. Heng</mark>, <mark>M. B. Davies</mark>, <mark>G. Olofsson</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| *In press. Accepted for publication in Astronomy and Astrophysics on 13 October 2022. 10 pages, 7 figures and 3 tables*|
|**Abstract**| Past occultation and phase-curve observations of the ultra-short period super-Earth 55 Cnc e obtained at visible and infrared wavelengths have been challenging to reconcile with a planetary reflection and emission model. In this study, we analyse a set of 41 occultations obtained over a two-year timespan with the CHEOPS satellite. We report the detection of 55 Cnc e's occultation with an average depth of $12\pm3$ ppm. We derive a corresponding 2-$\sigma$ upper limit on the geometric albedo of $A_g < 0.55$ once decontaminated from the thermal emission measured by Spitzer at 4.5$\mu$m. CHEOPS's photometric performance enables, for the first time, the detection of individual occultations of this super-Earth in the visible and identifies short-timescale photometric corrugations likely induced by stellar granulation. We also find a clear 47.3-day sinusoidal pattern in the time-dependent occultation depths that we are unable to relate to stellar noise, nor instrumental systematics, but whose planetary origin could be tested with upcoming JWST occultation observations of this iconic super-Earth. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.03747-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.03747) | **Monte Carlo Techniques for Addressing Large Errors and Missing Data in  Simulation-based Inference**  |
|| Bingjie Wang, Joel Leja, <mark>Ashley Villar</mark>, Joshua S. Speagle |
|*Appeared on*| *2022-11-08*|
|*Comments*| *8 pages, 2 figures, accepted to the Machine Learning and the Physical Sciences workshop at NeurIPS 2022*|
|**Abstract**| Upcoming astronomical surveys will observe billions of galaxies across cosmic time, providing a unique opportunity to map the many pathways of galaxy assembly to an incredibly high resolution. However, the huge amount of data also poses an immediate computational challenge: current tools for inferring parameters from the light of galaxies take $\gtrsim 10$ hours per fit. This is prohibitively expensive. Simulation-based Inference (SBI) is a promising solution. However, it requires simulated data with identical characteristics to the observed data, whereas real astronomical surveys are often highly heterogeneous, with missing observations and variable uncertainties determined by sky and telescope conditions. Here we present a Monte Carlo technique for treating out-of-distribution measurement errors and missing data using standard SBI tools. We show that out-of-distribution measurement errors can be approximated by using standard SBI evaluations, and that missing data can be marginalized over using SBI evaluations over nearby data realizations in the training set. While these techniques slow the inference process from $\sim 1$ sec to $\sim 1.5$ min per object, this is still significantly faster than standard approaches while also dramatically expanding the applicability of SBI. This expanded regime has broad implications for future applications to astronomical surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.02670-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.02670) | **Flashlights: More than A Dozen High-Significance Microlensing Events of  Extremely Magnified Stars in Galaxies at Redshifts z=0.7-1.5**  |
|| Patrick L. Kelly, et al. -- incl., <mark>Sung Kei Li</mark>, <mark>Jeremy Lim</mark>, <mark>Liliya L. R. Williams</mark>, <mark>Nathan Smith</mark> |
|*Appeared on*| *2022-11-08*|
|*Comments*| **|
|**Abstract**| Once only accessible in nearby galaxies, we can now study individual stars across much of the observable universe aided by galaxy-cluster gravitational lenses. When a star, compact object, or multiple such objects in the foreground galaxy-cluster lens become aligned, they can magnify a background individual star, and the timescale of a magnification peak can limit its size to tens of AU. The number and frequency of microlensing events therefore opens a window into the population of stars and compact objects, as well as high-redshift stars. To assemble the first statistical sample of stars in order to constrain the initial mass function (IMF) of massive stars at redshift z=0.7-1.5, the abundance of primordial black holes in galaxy-cluster dark matter, and the IMF of the stars making up the intracluster light, we are carrying out a 192-orbit program with the Hubble Space Telescope called "Flashlights," which is now two-thirds complete owing to scheduling challenges. We use the ultrawide F200LP and F350LP long-pass WFC3 UVIS filters and conduct two 16-orbit visits separated by one year. Having an identical roll angle during both visits, while difficult to schedule, yields extremely clean subtraction. Here we report the discovery of more than a dozen bright microlensing events, including multiple examples in the famous "Dragon Arc" discovered in the 1980s, as well as the "Spocks" and "Warhol" arcs that have hosted already known supergiants. The ultradeep observer-frame ultraviolet-through-optical imaging is sensitive to hot stars, which will complement deep James Webb Space Telescope infrared imaging. We are also acquiring Large Binocular Telescope LUCI and Keck-I MOSFIRE near-infrared spectra of the highly magnified arcs to constrain their recent star-formation histories. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.03641.md
    + _build/html/tmp_2211.03641/./fig/610.png
    + _build/html/tmp_2211.03641/././fig/CAMDs-dense.png
    + _build/html/tmp_2211.03641/./fig/lb.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\Unit}[1]{\ensuremath{\mathrm{~#1}}}$
$\newcommand{\pc}{\Unit{pc}}$
$\newcommand{\deriv}{\ensuremath{\mathrm{d}}}$
$\newcommand{\given}{\ensuremath{\hspace{0.07em}\mid\hspace{0.07em}}}$
$\newcommand{\parallax}{\ensuremath{\varpi}}$
$\newcommand{\parallaxsd}{\ensuremath{\sigma_\parallax}}$
$\newcommand{\parallaxtrue}{\ensuremath{\varpi_{\rm True}}}$
$\newcommand{\dist}{\ensuremath{r}}$
$\newcommand{\distest}{\ensuremath{\dist_{\rm est}}}$
$\newcommand{\distlen}{\ensuremath{L}}$
$\newcommand{\mg}{\ensuremath{M_{\rm G}}}$
$\newcommand{\gmag}{\ensuremath{G}}$
$\newcommand{\ag}{\ensuremath{A_{\rm G}}}$
$\newcommand{\com}[2]{ {\textcolor{orange}{ #1 :}}$
$    {\textcolor{red}{ #2}}}$
$\newcommand{\dt}[1]{{\tt #1}}$
$\newcommand{\secs}{\ensuremath{ \rm s}\xspace}$
$\newcommand{\ms}{\ensuremath{\rm  m s^{-1}}\xspace}$
$\newcommand{\kms}{\ensuremath{\rm  km s^{-1}}\xspace}$
$\newcommand{\masyr}{\ensuremath{\rm  mas yr^{-1}}\xspace}$
$\newcommand{\muas}{\ensuremath{ \mu\textnormal{as}}\xspace}$
$\newcommand{\muasyr}{\ensuremath{\muas \rm yr^{-1}}\xspace}$
$\newcommand{\inp}[1]{{\tt #1}}$
$\newcommand{\prog}[1]{{\it #1}}$
$\newcommand{\class}[1]{{\it #1}}$
$\newcommand{\pack}[1]{{\em #1}}$
$\newcommand{\property}[1]{{\em #1}}$
$\newcommand{\method}[1]{{\em #1}}$
$\newcommand{\file}[1]{{\em #1}}$
$\newcommand{\directory}[1]{{\em #1}}$
$\newcommand{\code}[1]{{\tt #1}}$
$\newcommand{\maspyr}{\muasyr}$
$\newcommand{\deriv}{\ensuremath{\mathrm{d}}}$
$\newcommand{\given}{\ensuremath{\hspace{0.1em}\mid\hspace{0.1em}}}$
$\newcommand{\prob}[2]{\ensuremath{{P^#1_#2}}}$
$\newcommand{\prior}[2]{\ensuremath{{\pi^#1_#2}}}$
$\newcommand{\bcg}{\ensuremath{\mathrm{BC}_G}\xspace}$
$\newcommand{\bcgsol}{\ensuremath{\mathrm{BC}_{G\odot}}\xspace}$
$\newcommand{\sigparallax}{\ensuremath{\sigma_{\varpi}}\xspace}$
$\newcommand{\dist}{\ensuremath{r}\xspace}$
$\newcommand{\aabun}{\ensuremath{[\alpha/\mathrm{H}]}\xspace}$
$\newcommand{\teff}{\ensuremath{T_{\mathrm{{eff}}}}\xspace}$
$\newcommand{\logteff}{\ensuremath{\log\teff}\xspace}$
$\newcommand{\lum}{\ensuremath{{L}}\xspace}$
$\newcommand{\loglum}{\ensuremath{\log\lum}\xspace}$
$\newcommand{\loggrav}{\ensuremath{\log g}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\mass}{\ensuremath{{M}}\xspace}$
$\newcommand{\radius}{\ensuremath{{R}}\xspace}$
$\newcommand{\Zabun}{\ensuremath{{Z}}\xspace}$
$\newcommand{\mh}{[M/H]\xspace}$
$\newcommand{\feh}{[Fe/H]\xspace}$
$\newcommand{\vsini}{\ensuremath{v\sin i}\xspace}$
$\newcommand{\azero}{\ensuremath{A_0}\xspace}$
$\newcommand{\ag}{\ensuremath{A_G}\xspace}$
$\newcommand{\abp}{\ensuremath{A_\mathrm{BP}}\xspace}$
$\newcommand{\arp}{\ensuremath{A_\mathrm{RP}}\xspace}$
$\newcommand{\av}{\ensuremath{A_\mathrm{V}}\xspace}$
$\newcommand{\rzero}{\ensuremath{R_0}\xspace}$
$\newcommand{\ebpminrp}{\ensuremath{E(G_{\rm BP} - G_{\rm RP})}\xspace}$
$\newcommand{\gmag}{\ensuremath{G}}$
$\newcommand{\bpmag}{\ensuremath{G_\mathrm{BP}}}$
$\newcommand{\rpmag}{\ensuremath{G_\mathrm{RP}}}$
$\newcommand{\mg}{M_\gmag}$
$\newcommand{\ra}{\ensuremath{\alpha}}$
$\newcommand{\dec}{\ensuremath{\delta}}$
$\newcommand{\parallax}{\ensuremath{\varpi}}$
$\newcommand{\sigparallax}{\ensuremath{\sigma_{\varpi}}}$
$\newcommand{\parzp}{\ensuremath{\varpi_{\rm zp}}}$
$\newcommand{\pmra}{\ensuremath{\mu_{\ra\ast}}}$
$\newcommand{\pmdec}{\ensuremath{\mu_\dec}}$
$\newcommand{\propm}{\ensuremath{\mu}}$
$\newcommand{\glon}{\ensuremath{\ell}}$
$\newcommand{\glat}{\ensuremath{b}}$
$\newcommand{\mag}{\ensuremath{ \mathrm{mag}}\xspace}$
$\newcommand{\nm}{\ensuremath{ \mathrm{nm}}\xspace}$
$\newcommand{\um}{\ensuremath{ \mu\mathrm{m}}\xspace}$
$\newcommand{\erg}{\ensuremath{ \rm erg}\xspace}$
$\newcommand{\ergs}{\ensuremath{ \rm erg~s}\xspace}$
$\newcommand{\yr}{\ensuremath{ \rm yr}\xspace}$
$\newcommand{\Myr}{\ensuremath{ \rm Myr}\xspace}$
$\newcommand{\Gyr}{\ensuremath{ \rm Gyr}\xspace}$
$\newcommand{\pc}{\ensuremath{ \rm pc}\xspace}$
$\newcommand{\kpc}{\ensuremath{ \rm kpc}\xspace}$
$\newcommand{\Mpc}{\ensuremath{ \rm Mpc}\xspace}$
$\newcommand{\Lsun}{\ensuremath{ {\lum}_{\odot}}\xspace}$
$\newcommand{\Msun}{\ensuremath{ {\mass}_{\odot}}\xspace}$
$\newcommand{\Rsun}{\ensuremath{ {\radius}_{\odot}}\xspace}$
$\newcommand{\Zsun}{\ensuremath{ {\Zabun}_{\odot}}\xspace}$
$\newcommand{\ang}{\ensuremath{ \text{\AA}}\xspace}$
$\newcommand{\mas}{\ensuremath{ \mathrm{mas}}\xspace}$
$\newcommand{\kms}{\ensuremath{\textrm{km s}^{-1}}}$
$\newcommand{\maspyr}{\ensuremath{\textrm{mas yr}^{-1}}}$
$\newcommand{\degree}{\ensuremath{^\circ}}$
$\newcommand{\hpix}{HEALPix}$
$\newcommand{\cairt}{\ion{Ca}{ii} IRT\xspace}$
$\newcommand{\mbolsun}{\ensuremath{M_{\rm bol}}\xspace}$
$\newcommand{\Tsun}{\ensuremath{{\teff_{\odot}}}\xspace}$
$\newcommand{\modulename}[1]{#1\xspace}$
$\newcommand{\apsis}{\modulename{Apsis}}$
$\newcommand{\smsgen}{\modulename{SMSGen}}$
$\newcommand{\dsc}{\modulename{DSC}}$
$\newcommand{\gspphot}{\modulename{GSP-Phot}}$
$\newcommand{\priam}{\modulename{Priam}}$
$\newcommand{\aeneas}{\modulename{Aeneas}}$
$\newcommand{\gspspec}{\modulename{GSP-Spec}}$
$\newcommand{\msc}{\modulename{MSC}}$
$\newcommand{\flame}{\modulename{FLAME}}$
$\newcommand{\espels}{\modulename{ESP-ELS}}$
$\newcommand{\esphs}{\modulename{ESP-HS}}$
$\newcommand{\espcs}{\modulename{ESP-CS}}$
$\newcommand{\espucd}{\modulename{ESP-UCD}}$
$\newcommand{\ugc}{\modulename{UGC}}$
$\newcommand{\oa}{\modulename{OA}}$
$\newcommand{\oca}{\modulename{OCA}}$
$\newcommand{\qsoc}{\modulename{QSOC}}$
$\newcommand{\tge}{\modulename{TGE}}$
$\newcommand{\mgalgo}{Matisse-Gauguin}$
$\newcommand{\basti}{BaSTI}$
$\newcommand$
$\newcommand{\gdr}[1]{Gaia~DR{#1}}$
$\newcommand{\gedr}[1]{Gaia~eDR{#1}}$
$\newcommand{\sample}[1]{sample~{#1}}$
$\newcommand{\tabname}[1]{\texttt{\detokenize{#1}}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\orange}{\textcolor{orange}}$
$\newcommand{\green}{\textcolor{OliveGreen}}$
$\newcommand{\XXX}{{~\red{XXX}~}}$
$\newcommand{\corr}[1]{\textbf{#1}}$
$\newcommand{\tick}{\checkmark}$
$\newcommand{\secref}[1]{Sect.~\ref{#1}}$
$\newcommand{\totalfields}{{538}\xspace}$
$\newcommand{\numberuniqueparameters}{{43}\xspace}$
$\newcommand{\linktodoc}{https://geapre.esac.esa.int/archive/documentation/GDR3}$
$\newcommand{\linktoapparam}[2]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_astrophysical_parameter_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}\xspace}}$
$\newcommand{\linktogalparam}[2]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_extra--galactic_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}\xspace}}$
$\newcommand{\linktogsparam}[2]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_main_source_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}\xspace}}$
$\newcommand{\linktotable}[1]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_astrophysical_parameter_tables/ssec_dm_#1.html}{\fieldName{#1}\xspace}}$
$\newcommand{\aptable}{\linktotable{astrophysical_parameters}}$
$\newcommand{\apsupptable}{\linktotable{astrophysical_parameters_supp}}$
$\newcommand{\linktogaltable}[1]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_extra--galactic_tables/ssec_dm_#1.html}{\fieldName{#1}\xspace}}$
$\newcommand{\linktogstable}[1]{\href{\linktodoc/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_#1.html}{\fieldName{#1}\xspace}}$
$\newcommand{\linksubsec}[2]{\href{\linktodoc/Data_analysis/chap_cu8par/#1.html}{#2\xspace}}$
$\newcommand{\linksec}[2]{\href{\linktodoc/Data_analysis/chap_cu8par/#1}{#2\xspace}}$
$\newcommand{\linkfig}[1]{\href{\linktodoc/Data_analysis/chap_cu8par/#1.html}{see table\xspace}}$
$\newcommand{\livelink}[2]{\href{https://dms.cosmos.esa.int/cs/livelink/open/#1}{\texttt{\textcolor{blue!70!black}{Livelink:#2}}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\azul}[1]{{\textcolor{blue}{#1}}}$
$\newcommand{\rojo}[1]{{\textcolor{red}{#1}}}$
$\newcommand{\bpminrp}{\ensuremath{G_\mathrm{BP}-G_\mathrm{RP}}\xspace}$
$\newcommand{\bporrp}{BP/RP\xspace}$
$\newcommand{\orcit}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand$
$\newcommand{\deg}{\ensuremath{^\circ}}$
$\newcommand{\arcmin}{\ensuremath{'}}$
$\newcommand{\arcsec}{\ensuremath{"}}$
$\newcommand{\fd}{\ensuremath{.\!\!^{\mathrm d}}}$
$\newcommand{\fh}{\ensuremath{.\!\!^{\mathrm h}}}$
$\newcommand{\fm}{\ensuremath{.\!\!^{\mathrm m}}}$
$\newcommand{\fs}{\ensuremath{.\!\!^{\mathrm s}}}$
$\newcommand{\fdg}{\ensuremath{.\!\!^\circ}}$
$\newcommand{\farcm}{\ensuremath{.\mkern-4mu^\prime}}$
$\newcommand{\farcs}{\ensuremath{.\!\!^{\prime\prime}}}$
$\newcommand{\extratrees}{\textsc{ExtraTrees}\xspace}$
$\newcommand{\@}{fieldName}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\Unit$}[1]{$\ensuremath${\mathrm{~#1}}}$
$\newcommand{$\pc$}{$\Unit${pc}}$
$\newcommand{$\deriv$}{$\ensuremath${\mathrm{d}}}$
$\newcommand{$\given$}{$\ensuremath${\hspace{0.07em}\mid\hspace{0.07em}}}$
$\newcommand{$\parallax$}{$\ensuremath${\varpi}}$
$\newcommand{$\parallax$sd}{$\ensuremath${\sigma_$\parallax$}}$
$\newcommand{$\parallax$true}{$\ensuremath${\varpi_{\rm True}}}$
$\newcommand{$\dist$}{$\ensuremath${r}}$
$\newcommand{$\dist$est}{$\ensuremath${$\dist$_{\rm est}}}$
$\newcommand{$\dist$len}{$\ensuremath${L}}$
$\newcommand{$\mg$}{$\ensuremath${M_{\rm G}}}$
$\newcommand{$\gmag$}{$\ensuremath${G}}$
$\newcommand{$\ag$}{$\ensuremath${A_{\rm G}}}$
$\newcommand{\com}[2]{ {\textcolor{orange}{ #1 :}}$
$    {\textcolor{red}{ #2}}}$
$\newcommand{$\dt$}[1]{{\tt #1}}$
$\newcommand{$\secs$}{$\ensuremath${ \rm s}$\xspace$}$
$\newcommand{$\ms$}{$\ensuremath${\rm  m s^{-1}}$\xspace$}$
$\newcommand{$\kms$}{$\ensuremath${\rm  km s^{-1}}$\xspace$}$
$\newcommand{$\masyr$}{$\ensuremath${\rm  mas yr^{-1}}$\xspace$}$
$\newcommand{$\muas$}{$\ensuremath${ \mu\textnormal{as}}$\xspace$}$
$\newcommand{$\muas$yr}{$\ensuremath${$\muas$ \rm yr^{-1}}$\xspace$}$
$\newcommand{$\inp$}[1]{{\tt #1}}$
$\newcommand{$\prog$}[1]{{\it #1}}$
$\newcommand{$\class$}[1]{{\it #1}}$
$\newcommand{$\pack$}[1]{{\em #1}}$
$\newcommand{$\property$}[1]{{\em #1}}$
$\newcommand{$\method$}[1]{{\em #1}}$
$\newcommand{$\file$}[1]{{\em #1}}$
$\newcommand{$\directory$}[1]{{\em #1}}$
$\newcommand{$\code$}[1]{{\tt #1}}$
$\newcommand{$\maspyr$}{$\muas$yr}$
$\newcommand{$\deriv$}{$\ensuremath${\mathrm{d}}}$
$\newcommand{$\given$}{$\ensuremath${\hspace{0.1em}\mid\hspace{0.1em}}}$
$\newcommand{$\prob$}[2]{$\ensuremath${{P^#1_#2}}}$
$\newcommand{$\prior$}[2]{$\ensuremath${{\pi^#1_#2}}}$
$\newcommand{$\bcg$}{$\ensuremath${\mathrm{BC}_G}$\xspace$}$
$\newcommand{$\bcg$sol}{$\ensuremath${\mathrm{BC}_{G\odot}}$\xspace$}$
$\newcommand{$\sigparallax$}{$\ensuremath${\sigma_{\varpi}}$\xspace$}$
$\newcommand{$\dist$}{$\ensuremath${r}$\xspace$}$
$\newcommand{$\aabun$}{$\ensuremath${[\alpha/\mathrm{H}]}$\xspace$}$
$\newcommand{$\teff$}{$\ensuremath${T_{\mathrm{{eff}}}}$\xspace$}$
$\newcommand{$\logteff$}{$\ensuremath${\log$\teff$}$\xspace$}$
$\newcommand{$\lum$}{$\ensuremath${{L}}$\xspace$}$
$\newcommand{$\loglum$}{$\ensuremath${\log$\lum$}$\xspace$}$
$\newcommand{$\loggrav$}{$\ensuremath${\log g}$\xspace$}$
$\newcommand{$\logg$}{$\ensuremath${\log g}$\xspace$}$
$\newcommand{$\mass$}{$\ensuremath${{M}}$\xspace$}$
$\newcommand{$\radius$}{$\ensuremath${{R}}$\xspace$}$
$\newcommand{$\Zabun$}{$\ensuremath${{Z}}$\xspace$}$
$\newcommand{$\mh$}{[M/H]$\xspace$}$
$\newcommand{$\feh$}{[Fe/H]$\xspace$}$
$\newcommand{$\vsini$}{$\ensuremath${v\sin i}$\xspace$}$
$\newcommand{$\azero$}{$\ensuremath${A_0}$\xspace$}$
$\newcommand{$\ag$}{$\ensuremath${A_G}$\xspace$}$
$\newcommand{$\abp$}{$\ensuremath${A_\mathrm{BP}}$\xspace$}$
$\newcommand{$\arp$}{$\ensuremath${A_\mathrm{RP}}$\xspace$}$
$\newcommand{$\av$}{$\ensuremath${A_\mathrm{V}}$\xspace$}$
$\newcommand{$\rzero$}{$\ensuremath${R_0}$\xspace$}$
$\newcommand{$\ebpminrp$}{$\ensuremath${E(G_{\rm BP} - G_{\rm RP})}$\xspace$}$
$\newcommand{$\gmag$}{$\ensuremath${G}}$
$\newcommand{$\bpmag$}{$\ensuremath${G_\mathrm{BP}}}$
$\newcommand{$\rpmag$}{$\ensuremath${G_\mathrm{RP}}}$
$\newcommand{$\mg$}{M_$\gmag$}$
$\newcommand{$\ra$}{$\ensuremath${\alpha}}$
$\newcommand{$\dec$}{$\ensuremath${\delta}}$
$\newcommand{$\parallax$}{$\ensuremath${\varpi}}$
$\newcommand{$\sigparallax$}{$\ensuremath${\sigma_{\varpi}}}$
$\newcommand{$\parzp$}{$\ensuremath${\varpi_{\rm zp}}}$
$\newcommand{$\pmra$}{$\ensuremath${\mu_{$\ra$\ast}}}$
$\newcommand{$\pmdec$}{$\ensuremath${\mu_$\dec$}}$
$\newcommand{$\propm$}{$\ensuremath${\mu}}$
$\newcommand{$\glon$}{$\ensuremath${\ell}}$
$\newcommand{$\glat$}{$\ensuremath${b}}$
$\newcommand{$\mag$}{$\ensuremath${ \mathrm{mag}}$\xspace$}$
$\newcommand{$\nm$}{$\ensuremath${ \mathrm{nm}}$\xspace$}$
$\newcommand{$\um$}{$\ensuremath${ \mu\mathrm{m}}$\xspace$}$
$\newcommand{$\erg$}{$\ensuremath${ \rm erg}$\xspace$}$
$\newcommand{$\erg$s}{$\ensuremath${ \rm erg~s}$\xspace$}$
$\newcommand{$\yr$}{$\ensuremath${ \rm yr}$\xspace$}$
$\newcommand{$\Myr$}{$\ensuremath${ \rm Myr}$\xspace$}$
$\newcommand{$\Gyr$}{$\ensuremath${ \rm Gyr}$\xspace$}$
$\newcommand{$\pc$}{$\ensuremath${ \rm pc}$\xspace$}$
$\newcommand{$\kpc$}{$\ensuremath${ \rm kpc}$\xspace$}$
$\newcommand{$\Mpc$}{$\ensuremath${ \rm Mpc}$\xspace$}$
$\newcommand{$\Lsun$}{$\ensuremath${ {$\lum$}_{\odot}}$\xspace$}$
$\newcommand{$\Msun$}{$\ensuremath${ {$\mass$}_{\odot}}$\xspace$}$
$\newcommand{$\Rsun$}{$\ensuremath${ {$\radius$}_{\odot}}$\xspace$}$
$\newcommand{$\Zsun$}{$\ensuremath${ {$\Zabun$}_{\odot}}$\xspace$}$
$\newcommand{$\ang$}{$\ensuremath${ \text{\AA}}$\xspace$}$
$\newcommand{$\mas$}{$\ensuremath${ \mathrm{mas}}$\xspace$}$
$\newcommand{$\kms$}{$\ensuremath${\textrm{km s}^{-1}}}$
$\newcommand{$\maspyr$}{$\ensuremath${\textrm{mas yr}^{-1}}}$
$\newcommand{$\degree$}{$\ensuremath${^\circ}}$
$\newcommand{$\hpix$}{HEALPix}$
$\newcommand{$\cairt$}{$\ion${Ca}{ii} IRT$\xspace$}$
$\newcommand{$\mbolsun$}{$\ensuremath${M_{\rm bol}}$\xspace$}$
$\newcommand{$\Tsun$}{$\ensuremath${{$\teff$_{\odot}}}$\xspace$}$
$\newcommand{$\modulename$}[1]{#1$\xspace$}$
$\newcommand{$\apsis$}{$\modulename${Apsis}}$
$\newcommand{$\smsgen$}{$\modulename${SMSGen}}$
$\newcommand{$\dsc$}{$\modulename${DSC}}$
$\newcommand{$\gspphot$}{$\modulename${GSP-Phot}}$
$\newcommand{$\priam$}{$\modulename${Priam}}$
$\newcommand{$\aeneas$}{$\modulename${Aeneas}}$
$\newcommand{$\gspspec$}{$\modulename${GSP-Spec}}$
$\newcommand{$\ms$c}{$\modulename${MSC}}$
$\newcommand{$\flame$}{$\modulename${FLAME}}$
$\newcommand{$\espels$}{$\modulename${ESP-ELS}}$
$\newcommand{$\esphs$}{$\modulename${ESP-HS}}$
$\newcommand{$\espcs$}{$\modulename${ESP-CS}}$
$\newcommand{$\espucd$}{$\modulename${ESP-UCD}}$
$\newcommand{$\ugc$}{$\modulename${UGC}}$
$\newcommand{$\oa$}{$\modulename${OA}}$
$\newcommand{$\oca$}{$\modulename${OCA}}$
$\newcommand{$\qsoc$}{$\modulename${QSOC}}$
$\newcommand{$\tge$}{$\modulename${TGE}}$
$\newcommand{$\mg$algo}{Matisse-Gauguin}$
$\newcommand{$\basti$}{BaSTI}$
$\newcommand$
$\newcommand{$\gdr$}[1]{Gaia~DR{#1}}$
$\newcommand{$\gedr$}[1]{Gaia~eDR{#1}}$
$\newcommand{$\sample$}[1]{sample~{#1}}$
$\newcommand{$\tabname$}[1]{\texttt{\detokenize{#1}}}$
$\newcommand{$\red$}{\textcolor{red}}$
$\newcommand{$\orange$}{\textcolor{orange}}$
$\newcommand{$\green$}{\textcolor{OliveGreen}}$
$\newcommand{$\XXX$}{{~$\red${XXX}~}}$
$\newcommand{$\corr$}[1]{\textbf{#1}}$
$\newcommand{$\tick$}{\checkmark}$
$\newcommand{$\secref$}[1]{Sect.~\ref{#1}}$
$\newcommand{$\totalfields$}{{538}$\xspace$}$
$\newcommand{$\numberuniqueparameters$}{{43}$\xspace$}$
$\newcommand{$\linktodoc$}{https://geapre.esac.esa.int/archive/documentation/GDR3}$
$\newcommand{$\linktoapparam$}[2]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_astrophysical_parameter_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}$\xspace$}}$
$\newcommand{$\linktogalparam$}[2]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_extra--galactic_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}$\xspace$}}$
$\newcommand{$\linktogsparam$}[2]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_main_source_tables/ssec_dm_#1.html\##1-#2}{\fieldName{#2}$\xspace$}}$
$\newcommand{$\linktotable$}[1]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_astrophysical_parameter_tables/ssec_dm_#1.html}{\fieldName{#1}$\xspace$}}$
$\newcommand{$\aptable$}{$\linktotable${astrophysical_parameters}}$
$\newcommand{$\apsupptable$}{$\linktotable${astrophysical_parameters_supp}}$
$\newcommand{$\linktogaltable$}[1]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_extra--galactic_tables/ssec_dm_#1.html}{\fieldName{#1}$\xspace$}}$
$\newcommand{$\linktogstable$}[1]{\href{$\linktodoc$/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_#1.html}{\fieldName{#1}$\xspace$}}$
$\newcommand{$\linksubsec$}[2]{\href{$\linktodoc$/Data_analysis/chap_cu8par/#1.html}{#2$\xspace$}}$
$\newcommand{$\linksec$}[2]{\href{$\linktodoc$/Data_analysis/chap_cu8par/#1}{#2$\xspace$}}$
$\newcommand{$\linkfig$}[1]{\href{$\linktodoc$/Data_analysis/chap_cu8par/#1.html}{see table$\xspace$}}$
$\newcommand{$\livelink$}[2]{\href{https://dms.cosmos.esa.int/cs/livelink/open/#1}{\texttt{\textcolor{blue!70!black}{Livelink:#2}}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\azul$}[1]{{\textcolor{blue}{#1}}}$
$\newcommand{$\rojo$}[1]{{\textcolor{red}{#1}}}$
$\newcommand{$\bpminrp$}{$\ensuremath${G_\mathrm{BP}-G_\mathrm{RP}}$\xspace$}$
$\newcommand{$\bporrp$}{BP/RP$\xspace$}$
$\newcommand{$\orcit$}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$
$\newcommand$
$\newcommand{$\deg$}{$\ensuremath${^\circ}}$
$\newcommand{$\arcmin$}{$\ensuremath${'}}$
$\newcommand{$\arcsec$}{$\ensuremath${"}}$
$\newcommand{$\fd$}{$\ensuremath${.\!\!^{\mathrm d}}}$
$\newcommand{$\fh$}{$\ensuremath${.\!\!^{\mathrm h}}}$
$\newcommand{$\fm$}{$\ensuremath${.\!\!^{\mathrm m}}}$
$\newcommand{$\fs$}{$\ensuremath${.\!\!^{\mathrm s}}}$
$\newcommand{$\fd$g}{$\ensuremath${.\!\!^\circ}}$
$\newcommand{$\farcm$}{$\ensuremath${.\mkern-4mu^\prime}}$
$\newcommand{$\farcs$}{$\ensuremath${.\!\!^{\prime\prime}}}$
$\newcommand{$\extratrees$}{$\textsc${ExtraTrees}$\xspace$}$
$\newcommand{\@}{fieldName}$</div>



<div id="title">

#  DR3

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.03641-b31b1b.svg)](https://arxiv.org/abs/2211.03641)<mark>Appeared on: 2022-11-08</mark> - _Accepted by Astronomy and Astrophysics. 29 pages, 20 figures plus 3 appendices_

</div>
<div id="authors">

L.M.                          Sarro, et al. -- incl., <mark>M.              García-Torres</mark>, <mark>C.A.L.                   Bailer-Jones</mark>, <mark>R.                         Andrae</mark>, <mark>M.                      Fouesneau</mark>, <mark>E.                        Livanou</mark>, <mark>H.                           Zhao</mark>

</div>
<div id="abstract">

**Abstract:** Previous\gaiaData Releases offered the opportunity to uncover ultracool dwarfs (UCDs) through astrometric, rather than purely photometric selection. The most recent, third data release offers in addition the opportunity to use low-resolution spectra to refine and widen the selection.In this work we use the\gaiaDR3 set of ultracool dwarf candidates and complement the\gaiaspectrophotometry with additional photometry in order to characterise its global properties. This includes the inference of the distances, their locus in the\gaiacolour-absolute magnitude diagram and the (biased through selection) luminosity function in the faint end of the Main Sequence. We study the overall changes in the\gaiaRP spectra as a function of spectral type. We study the UCDs in binary systems, attempt to identify low-mass members of nearby young associations, star forming regions and clusters, and analyse their variability properties.We use a forward model and the Bayesian inference framework to produce posterior probabilities for the distribution parameters and a calibration of the colour index as a function of the absolute magnitude in the form of a Gaussian Process. Additionally we apply the HMAC unsupervised classification algorithm for the detection and characterisation of overdensities in the space of celestial coordinates, projected velocities and parallaxes.We detect 57 young, kinematically homogeneous groups some of which are identified as well known star forming regions, associations and clusters of different ages. We find that the primary members of 880 binary systems with a UCD belong mainly to the thin and thick disk components of the Milky Way. We identify 1109 variable UCDs using the variability tables in the\gaiaarchive, 728 of which belong to the star forming regions defined by HMAC. We define two groups of variable UCDs with extreme bright or faint outliers.The set of sources identified as UCDs in the\gaiaarchive contains a wealth of information that will require focused follow-up studies and observations. It will help to advance our understanding of the nature of the faint end of the Main Sequence and the stellar/substellar transition.

</div>

<div id="div_fig1">

<img src="tmp_2211.03641/./fig/610.png" alt="Fig46" width="100%"/>

**Figure 46. -** Properties of one (level 4) HMAC cluster tentatively
	        identified with Serpens (but see text) with no sources
	        in common with BANYAN associations. From left to
	        right: equatorial coordinates with various colours
	        separating subclusters at a hierarchy level lower (and
	        hence finer) than 4; CAMD (points represent the full
	        set of UCD candidates in the catalogue and circles
	        identify the HMAC group at level 4; black dots
	        represent sources not assigned to any subcluster in
	        the lower level); tangential velocities; and the
	        stacked histogram of the inverse of the parallax in
	        all subclusters. (*fig:hmac-only-610*)

</div>
<div id="div_fig2">

<img src="tmp_2211.03641/././fig/CAMDs-dense.png" alt="Fig39" width="100%"/>

**Figure 39. -** Colour-absolute magnitude diagrams (CAMDs) combining several \gaia, 2MASS and WISE magnitudes. The top row represents all sources with available measurements and the bottom row shows the subset with quality flags A or B in the corresponding 2MASS and AllWISE catalogues. A kernel density estimate is shown using a grey scale. The transparency and the symbol sizes were chosen to enhance visibility of the main densities. The colour code reflects the estimated $\teff$ as indicated by the colour bar at the lower right edge of the Figure. (*fig:camds-dense*)

</div>
<div id="div_fig3">

<img src="tmp_2211.03641/./fig/lb.png" alt="Fig41" width="100%"/>

**Figure 41. -** Distribution in Galactic coordinates of the sources in quality classes 0 and 1 (upper left) and 2 (upper right) of the \gaia UCD catalogue using the aitoff projection. Lower left panel: clusters at level 4 of the hierarchy with more than 10 members identified by HMAC in the set of sources in quality classes 0 and 1. There are 88 such clusters, the largest 13 of which are identified with prominent colours and the rest are plotted in salmon for clarity. Lower right panel: the rest of the sources in clusters with 10 members or less. We mark in red and blue the two most prominent overdensities. (*fig:lb*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

83  publications in the last 7 days.
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers